In [1]:
%matplotlib inline
import numpy as np
import torch.nn as nn
import torch.nn.functional as F
import matplotlib.pyplot as plt
from plots import *
from kls import *
from torch.distributions.normal import Normal
from torch.distributions.one_hot_categorical import OneHotCategorical as cat
from torch.distributions.beta import Beta
from torch.distributions.uniform import Uniform
from torch.distributions.relaxed_categorical import RelaxedOneHotCategorical as rcat
from torch import logsumexp
import sys
import time
import datetime
import math

In [2]:
Xs = torch.from_numpy(np.load('crosses/obs.npy')).float()
Mus = torch.from_numpy(np.load('crosses/mus.npy')).float()
Xs = Xs.transpose(1,2)

B, N, D = Xs.shape
## Model Parametersnum_hidden
NUM_SAMPLES = 10
NUM_HIDDEN = 32
NUM_LATENTS = 2
NUM_OBS = D
BATCH_SIZE = 100
NUM_EPOCHS = 20000
LEARNING_RATE = 1e-4
CUDA = False
PATH = 'cross'

In [3]:
class Encoder(nn.Module):
    def __init__(self, num_obs= D,
                       num_hidden=NUM_HIDDEN,
                       num_latents=NUM_LATENTS):
        super(self.__class__, self).__init__()
        self.enc_h = nn.Sequential(
            nn.Linear(num_obs, num_hidden),
            nn.Tanh())
        self.enc_log_u_a = nn.Sequential(
            nn.Linear(num_hidden, int(0.5*num_hidden)),
            nn.Tanh(),
            nn.Linear(int(0.5*num_hidden), num_latents))
        self.enc_log_u_b = nn.Sequential(
            nn.Linear(num_hidden, int(0.5*num_hidden)),
            nn.Tanh(),
            nn.Linear(int(0.5*num_hidden), num_latents))

    def forward(self, obs, num_samples):
        h = self.enc_h(obs) # (B, H)
        a = torch.exp(self.enc_log_u_a(h)) # (B, 2)
        b = torch.exp(self.enc_log_u_b(h)) # (B, 2)
        q_u = Beta(a, b)
        u = q_u.sample((num_samples,)) # (S, B, 2)
        log_q_u = q_u.log_prob(u).sum(-1)
        return u, log_q_u
    
    
class Decoder(nn.Module):
    def __init__(self, num_obs=NUM_OBS,
                       num_hidden=NUM_HIDDEN,
                       num_latents=NUM_LATENTS):
        super(self.__class__, self).__init__()

        self.x_sigma = 0.01 * torch.ones((N, num_obs))
        self.dec_mu = nn.Sequential(
            nn.Linear(num_latents, num_hidden),
            nn.Tanh(),
            nn.Linear(num_hidden, num_obs))

    def forward(self, u, obs, num_samples):
        mu = self.dec_mu(u) # (S, B, K+1) -> (S, B, O)
        p_x = Normal(mu, self.x_sigma.repeat(num_samples, 1, 1)) # (S, B, O)
        log_p_x = p_x.log_prob(obs).sum(-1) # (S, B)
        return mu, log_p_x

In [4]:
def weights_init(m):
    classname = m.__class__.__name__
    if classname.find('Linear') != -1:
        m.weight.data.normal_(0.0, 1e-2)     
        
def initialize():
    enc = Encoder()
    dec = Decoder()
#     enc.apply(weights_init)
    opt1 =  torch.optim.Adam(list(enc.parameters()),lr=LEARNING_RATE, betas=(0.9, 0.99)) 
    opt2 = torch.optim.Adam(list(dec.parameters()),lr=LEARNING_RATE, betas=(0.9, 0.99))
    return enc, dec, opt1, opt2
enc, dec, opt1, opt2 = initialize()

In [5]:
def oneshot(x, mean, N, D, num_samples):
    data = x - mean
    X_mus = torch.zeros((num_samples, N, D))
    u, log_q = enc(data, num_samples)
    mu, log_p_x = dec(u, data, num_samples)
    pr_u = Uniform(torch.zeros(N, NUM_LATENTS), torch.ones(N, NUM_LATENTS))
    log_pr_u = pr_u.log_prob(u).sum(-1)
    log_weights = (log_pr_u + log_p_x - log_q)
    weights = torch.exp(log_weights - logsumexp(log_weights, 0)).detach()
    eubo = torch.mul(weights, log_weights).sum(0).mean()
    elbo = log_weights.mean(0).mean()

    return eubo, elbo, mu, u

In [6]:
ELBOs = []
EUBOs = []
time_start = time.time()

for epoch in range(NUM_EPOCHS):
    for b in range(B):
        X = Xs[b]
        mean = Mus[b]
        indices = torch.randperm(N)
        X_shuffle = X[indices]
        opt1.zero_grad()
        eubo, elbo, mu, u = oneshot(X_shuffle, mean, N, D, NUM_SAMPLES)
        eubo.backward()
        opt1.step()
        opt2.zero_grad()
        eubo, elbo, mu, u = oneshot(X_shuffle, mean, N, D, NUM_SAMPLES)
        (-elbo).backward()
        opt2.step()
    
        ELBOs.append(elbo.item())
        EUBOs.append(eubo.item())
    

    time_end = time.time()  
    print('epoch=%d, EUBO=%.3f, ELBO=%.3f (%ds)' % (epoch, eubo, elbo, time_end - time_start))
    time_start = time.time()


epoch=0, EUBO=-1367.802, ELBO=-1981.988 (1s)
epoch=1, EUBO=-1333.350, ELBO=-1871.084 (1s)
epoch=2, EUBO=-1266.404, ELBO=-1780.592 (1s)
epoch=3, EUBO=-1135.305, ELBO=-1671.246 (1s)
epoch=4, EUBO=-1001.174, ELBO=-1569.007 (1s)
epoch=5, EUBO=-832.875, ELBO=-1385.950 (1s)
epoch=6, EUBO=-704.228, ELBO=-1239.736 (1s)
epoch=7, EUBO=-583.007, ELBO=-1081.367 (1s)
epoch=8, EUBO=-484.730, ELBO=-936.657 (1s)
epoch=9, EUBO=-391.564, ELBO=-805.074 (1s)
epoch=10, EUBO=-323.066, ELBO=-710.885 (1s)
epoch=11, EUBO=-259.736, ELBO=-595.729 (1s)
epoch=12, EUBO=-220.280, ELBO=-531.473 (1s)
epoch=13, EUBO=-178.409, ELBO=-447.522 (1s)
epoch=14, EUBO=-137.408, ELBO=-367.276 (1s)
epoch=15, EUBO=-110.044, ELBO=-308.066 (1s)
epoch=16, EUBO=-78.959, ELBO=-234.489 (1s)
epoch=17, EUBO=-61.097, ELBO=-197.748 (1s)
epoch=18, EUBO=-43.806, ELBO=-154.006 (1s)
epoch=19, EUBO=-31.143, ELBO=-130.554 (1s)
epoch=20, EUBO=-22.106, ELBO=-101.227 (1s)
epoch=21, EUBO=-15.794, ELBO=-84.407 (1s)
epoch=22, EUBO=-10.765, ELBO=-67.351

KeyboardInterrupt: 

In [7]:
torch.save(enc.state_dict(), 'VAE/enc-cross')
torch.save(dec.state_dict(), 'VAE/dec-cross')

In [9]:
eubo, elbo, mu, u = oneshot(Xs,mean,  N, D, NUM_SAMPLES)

RuntimeError: The size of tensor a (100) must match the size of tensor b (10) at non-singleton dimension 1

In [ ]:
T = 40
fig = plt.figure(figsize=(12,5))
ax1 = fig.add_subplot(1,2,1)
ax2 = fig.add_subplot(1,2,2)     

ax1.scatter(Xs[:T, 0].data.numpy(), Xs[:T, 1].data.numpy())
ax1.set_xlim([-3,3])
ax1.set_ylim([-3,3])
ax1.set_title('data')

uu = u.mean(0)
ax2.plot(uu[:T, 0].data.numpy(), 'go')
ax2.plot(uu[:T, 1].data.numpy(), 'go')

In [ ]:
T = 150
plt.scatter(Xs[:T, 0].data.numpy(), Xs[:T, 1].data.numpy())
plt.xlim([-3,3])
plt.ylim([-3,3])

In [ ]:
for s in range(NUM_SAMPLES):
    fig = plt.figure(figsize=(12,5))
    ax1 = fig.add_subplot(1,2,1)
    ax2 = fig.add_subplot(1,2,2)   
    ax1.scatter(mu[s, :, 0].data.numpy(), mu[s, :, 1].data.numpy())
#     ax2.scatter(Xs_shuffle[:, 0].data.numpy(), Xs_shuffle[:, 1].data.numpy())
    ax1.set_xlim([-3,3])
    ax1.set_ylim([-3,3])


In [ ]:
a = torch.linspace(0.0, 0.2, 100)[:, None]
angles = dec.dec_angles(a) 
rads = dec.dec_rad(a)
x_mus = angles * rads

fig = plt.figure(figsize=(5,5))
ax1 = fig.add_subplot(1,1,1)
ax1.scatter(x_mus[:, 0].data.numpy(), x_mus[:, 1].data.numpy())
ax1.set_xlim([-3,3])
ax1.set_ylim([-3,3])